In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Utils
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, make_scorer, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold
import lightgbm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise  import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel
from catboost import CatBoostClassifier
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
from sklearn.metrics import log_loss
from sklearn.preprocessing import KBinsDiscretizer
import category_encoders as ce
from tpot import TPOTClassifier
import timeit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from tpot.builtins import ZeroCount
from sklearn.preprocessing import RobustScaler

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
sub = pd.read_csv('/kaggle/input/submission/submission (3).csv', sep=',')

In [ ]:
sub.to_csv('sub.csv')

## Load Data

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv', sep=',')
train = train.set_index('id')

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv', sep=',')
test = test.set_index('id')

In [ ]:
# df for psudo-labeling LB 1.08631
pseudo_df = pd.read_csv('/kaggle/input/pseudo-df/submission (7).csv', sep=',')
pseudo_df = pseudo_df.set_index('id')

In [ ]:
fig = px.histogram(train, x="target",
                   width=600, 
                   height=400,
                   histnorm='percent',
                   template="simple_white"
                   )
fig.update_layout(title="Target Description", 
                  font_family="San Serif",
                  titlefont={'size': 20},
                  showlegend=True,
                  legend=dict(
                      orientation="v", 
                      y=1, 
                      yanchor="top", 
                      x=1.0, 
                      xanchor="right"
                  )                
                 ).update_xaxes(categoryorder='total descending')#
fig.update_traces( 
                  marker_line_width=1.5, opacity=0.99)
fig.show()

In [ ]:
cols = train.columns[:-1]
target = train['target']
train = train[cols]

### MLJAR multiclass Classifier

##### Install and Import

In [ ]:
!pip install -q -U matplotlib==3.1.3

In [ ]:
!pip install -q -U git+https://github.com/mljar/mljar-supervised.git@master

In [ ]:
from supervised.automl import AutoML

#### Model Definition

Given the good results obtained with Three based model, I'll use a custom model suggested in the documentation
from: documentation https://supervised.mljar.com/features/modes/


In [ ]:
automl = AutoML(
    mode = "Compete",
    algorithms=["CatBoost", "Xgboost"],
    total_time_limit=4*3600,
    start_random_models=10,
    hill_climbing_steps=4,
    top_models_to_improve=3,
    golden_features=False,
    features_selection=False,
    stack_models=True,
    train_ensemble=True,
    explain_level=0,
    ml_task = 'multiclass_classification',
    eval_metric='logloss',
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 10,
        "shuffle": False,
        "stratify": True,
    }
)

### Training 

In [ ]:
automl.fit(train, target)

### Simple Report with Training output

In [ ]:
automl.report()

### Load Submission Sample 

In [ ]:
sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv', sep=',')
sub_sample = sub_sample.set_index('id')

### Predictions

In [ ]:
predictions = automl.predict_all(test)

In [ ]:
oof_predictions = automl.predict_all(train)

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv', sep=',')
test = test.set_index('id')

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv', sep=',')
train = train.set_index('id')

In [ ]:
pred= predictions.set_index(test.index)

In [ ]:
oof = oof_predictions.set_index(train.index)

In [ ]:
oof = oof.reset_index()

In [ ]:
pred.head()

In [ ]:
p = pred['label']

In [ ]:
p = p.reset_index()

In [ ]:
p.to_csv('p.csv',index=False)

In [ ]:
oof.to_csv('oof.csv', index=False)

In [ ]:
sub_sample['Class_1'] = np.clip(pred['prediction_Class_1'],0.02, 0.98)
sub_sample['Class_2'] = np.clip(pred['prediction_Class_2'],0.02, 0.98)
sub_sample['Class_3'] = np.clip(pred['prediction_Class_3'],0.02, 0.98)
sub_sample['Class_4'] = np.clip(pred['prediction_Class_4'],0.02, 0.98)

In [ ]:
pred['prediction_Class_1'].head()

In [ ]:
sub_sample.head()

In [ ]:
sub_sample = sub_sample.reset_index()

In [ ]:
sub_sample.to_csv('submission.csv',index=False)